In [1]:
import sys
!conda install --yes --prefix {sys.prefix} boto3
!conda install --yes --prefix {sys.prefix} botocore

import boto3
import botocore
from botocore import UNSIGNED
from botocore.client import Config

Solving environment: done

# All requested packages already installed.

Solving environment: done

# All requested packages already installed.



In [2]:
sys.argv = ['download_S3_files.py', 'tcc-dataset-wifi-fingerprints', '', 'imports/', '/projects/datasets/TCC/']

bucket = sys.argv[1]
key = sys.argv[2]
prefix = sys.argv[3]
store_path = sys.argv[4]

if bucket is None or key is None or prefix is None or store_path is None:
    print("bucket, key, prefix and store_path params are required")
    
s3_conn   = boto3.client('s3', config=Config(signature_version=UNSIGNED))
s3_result =  s3_conn.list_objects_v2(Bucket=bucket, Prefix=prefix, Delimiter = "/")
files = []

if 'Contents' not in s3_result:
        files = []
else:
    for key in s3_result['Contents']:
        files.append(key['Key'])
    
    while s3_result['IsTruncated']:
        continuation_key = s3_result['NextContinuationToken']
        s3_result = s3_conn.list_objects_v2(Bucket=bucket, Prefix=prefix, Delimiter="/", ContinuationToken=continuation_key)
        for key in s3_result['Contents']:
            files.append(key['Key'])
print files

[u'imports/02-12-daniel.json', u'imports/02-12-raspberry-3.json', u'imports/02-12-raspberry-henrique.json', u'imports/A-E-extended.json', u'imports/A-E-part2.json', u'imports/A-E-part3.json', u'imports/A-E.json', u'imports/A107.json', u'imports/A113.json', u'imports/A114.json', u'imports/A115.json', u'imports/A116.json', u'imports/A117.json', u'imports/A118-extended.json', u'imports/A118-part2.json', u'imports/A118.json', u'imports/A119-extended.json', u'imports/A119-part2.json', u'imports/A119.json', u'imports/A120 copy.json', u'imports/A120-extended.json', u'imports/A120-part2.json', u'imports/A120-part3.json', u'imports/A120.json', u'imports/A121-extended.json', u'imports/A121-part2.json', u'imports/A121.json', u'imports/A123-extended.json', u'imports/A123-part2.json', u'imports/A123.json', u'imports/A125-extended.json', u'imports/A125-part2.json', u'imports/A125.json', u'imports/B-A-extended.json', u'imports/B-A-part2.json', u'imports/B-A.json', u'imports/B126-extended.json', u'imp

In [5]:
s3 = boto3.resource('s3', config=Config(signature_version=UNSIGNED))

try:
    for file in files:
        print('Downloading ' + file + '...')
        
        s3.Bucket(bucket).download_file(file, store_path + file)
except botocore.exceptions.ClientError as e:
    if e.response['Error']['Code'] == '404':
        print('The object does not exist.')
    else:
        raise